In [42]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz


FOLDER_ID = os.getenv("FOLDER_ID") # id каталога из которого береться запрос
FOLDER = os.getenv("FOLDER") #имя папки в которую будет выкладываться
ACCESS_KEY = os.getenv("ACCESS_KEY") #aws_access_key_id для S3
SECRET_KEY = os.getenv("SECRET_KEY") #aws_secret_access_key в s3
BUCKET_NAME = os.getenv("BUCKET_NAME") #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "C:/Users/Boris/Documents/GitHub/analytics3/functions/support-requests-data 2/temp_file"
URL_ALERT = os.getenv("URL_ALERT")

url = 'https://docs.google.com/spreadsheets/d/1tuVDy6kU6KeyoGNIpoGRhRuuN5Mc165mijqUA0j83Og/gviz/tq?tqx=out:csv&sheet=form1'

response = requests.get(url)




In [17]:
data = response.content
with open(TEMP_FILENAME, 'wb') as file:
    file.write(data)
    

In [35]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd

FOLDER_ID = os.getenv("FOLDER_ID") # id каталога из которого береться запрос
FOLDER = os.getenv("FOLDER") #имя папки в которую будет выкладываться
ACCESS_KEY = os.getenv("ACCESS_KEY") #aws_access_key_id для S3
SECRET_KEY = os.getenv("SECRET_KEY") #aws_secret_access_key в s3
BUCKET_NAME = os.getenv("BUCKET_NAME") #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "/tmp/temp_file.csv"
URL_ALERT = os.getenv("URL_ALERT")

def get_s3_instance(): # функция создает соединение
        session = boto3.session.Session()
        return session.client(
            aws_access_key_id=ACCESS_KEY,
            aws_secret_access_key=SECRET_KEY,
            service_name='s3',
            endpoint_url='https://storage.yandexcloud.net'
        )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=f"{FOLDER}/support_requests_data_partner.csv"
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)





In [43]:
df = pd.read_csv(url)

df['ID устройства'] = df['ID устройства'].astype(str)
df['ID устройства'] = df['ID устройства'].apply(lambda x: x.split(sep=','))
df = df.explode('ID устройства')
df = df.reset_index().drop('index', axis=1)

In [44]:
df

,Отметка времени,ID устройства,[Это авария],Обращение по:
0,19.09.2024 14:48:49,123456789,Нет,"Железо, тест"
1,19.09.2024 15:02:16,2235142860,NaN,МП
2,19.09.2024 15:02:31,2018115131,NaN,Железо
3,19.09.2024 15:29:26,2,NaN,МП
4,19.09.2024 15:29:26,12513E+19,NaN,МП
...,...,...,...,...
5447,17.10.2024 12:00:20,2236151580,Нет,Обслуживающая организация
5448,17.10.2024 12:05:12,2428253294,Нет,"Видео, user"
5449,17.10.2024 12:31:59,2329250131,Нет,Видео
5450,17.10.2024 13:13:29,2208135221,Нет,МП


In [45]:
df['Обращение по:'] = df['Обращение по:'].astype(str)
df['Обращение по:'] = df['Обращение по:'].apply(lambda x: x.split(sep=','))
df = df.explode('Обращение по:')
df = df.reset_index().drop('index', axis=1)
df

,Отметка времени,ID устройства,[Это авария],Обращение по:
0,19.09.2024 14:48:49,123456789,Нет,Железо
1,19.09.2024 14:48:49,123456789,Нет,тест
2,19.09.2024 15:02:16,2235142860,NaN,МП
3,19.09.2024 15:02:31,2018115131,NaN,Железо
4,19.09.2024 15:29:26,2,NaN,МП
...,...,...,...,...
5851,17.10.2024 12:05:12,2428253294,Нет,Видео
5852,17.10.2024 12:05:12,2428253294,Нет,user
5853,17.10.2024 12:31:59,2329250131,Нет,Видео
5854,17.10.2024 13:13:29,2208135221,Нет,МП


In [46]:
list_of_columns = ['ID устройства', 'Обращение по:']
for i in list_of_columns:
    df[i] = df[i].fillna('').astype(str).apply(lambda x:x.split(','))
    df = df.explode(i)
    df[i] = df[i].apply(lambda x:x.strip())

In [47]:
df

,Отметка времени,ID устройства,[Это авария],Обращение по:
0,19.09.2024 14:48:49,123456789,Нет,Железо
1,19.09.2024 14:48:49,123456789,Нет,тест
2,19.09.2024 15:02:16,2235142860,NaN,МП
3,19.09.2024 15:02:31,2018115131,NaN,Железо
4,19.09.2024 15:29:26,2,NaN,МП
...,...,...,...,...
5851,17.10.2024 12:05:12,2428253294,Нет,Видео
5852,17.10.2024 12:05:12,2428253294,Нет,user
5853,17.10.2024 12:31:59,2329250131,Нет,Видео
5854,17.10.2024 13:13:29,2208135221,Нет,МП


In [36]:

    df['Обращение по:'] = df['Обращение по:'].astype(str)
    df['Обращение по:'] = df['Обращение по:'].apply(lambda x: x.split(sep=','))
    df = df.explode('Обращение по:')
    df = df.reset_index().drop('index', axis=1)
    list_of_columns = ['ID устройства', 'Обращение по:']
    for i in list_of_columns:
        df[i] = df[i].fillna('').astype(str).apply(lambda x:x.split(','))
        df = df.explode(i)
        df[i] = df[i].apply(lambda x:x.strip())

    df.to_csv(TEMP_FILENAME) 
    get_s3_instance()
    upload_dump_to_s3()
    remove_temp_files()


,Отметка времени,ID устройства,[Это авария],Обращение по:
0,19.09.2024 14:48:49,123456789,Нет,"Железо, тест"
1,19.09.2024 15:02:16,2235142860,NaN,МП
2,19.09.2024 15:02:31,2018115131,NaN,Железо
3,19.09.2024 15:29:26,"2,12513E+19",NaN,МП
4,19.09.2024 15:35:39,2330153753,NaN,Прошивка домофона
...,...,...,...,...
5331,17.10.2024 12:00:20,2236151580,Нет,Обслуживающая организация
5332,17.10.2024 12:05:12,2428253294,Нет,"Видео, user"
5333,17.10.2024 12:31:59,2329250131,Нет,Видео
5334,17.10.2024 13:13:29,2208135221,Нет,МП


In [37]:
df['ID устройства'] = df['ID устройства'].astype(str)
df['ID устройства'] = df['ID устройства'].apply(lambda x: x.split(sep=','))
df = df.explode('ID устройства')
df = df.reset_index().drop('index', axis=1)
df['Обращение по:'] = df['Обращение по:'].astype(str)
df['Обращение по:'] = df['Обращение по:'].apply(lambda x: x.split(sep=','))
df = df.explode('Обращение по:')
df = df.reset_index().drop('index', axis=1)
list_of_columns = ['ID устройства', 'Обращение по:']
for i in list_of_columns:
    df[i] = df[i].fillna('').astype(str).apply(lambda x:x.split(','))
    df = df.explode(i)
    df[i] = df[i].apply(lambda x:x.strip())

df.to_csv(TEMP_FILENAME) 
get_s3_instance()
upload_dump_to_s3()
remove_temp_files()
requests.get(os.getenv("URL_ALERT"))

,Отметка времени,ID устройства,[Это авария],Обращение по:
0,19.09.2024 14:48:49,123456789,Нет,Железо
1,19.09.2024 14:48:49,123456789,Нет,тест
2,19.09.2024 15:02:16,2235142860,NaN,МП
3,19.09.2024 15:02:31,2018115131,NaN,Железо
4,19.09.2024 15:29:26,2,NaN,МП
...,...,...,...,...
5851,17.10.2024 12:05:12,2428253294,Нет,Видео
5852,17.10.2024 12:05:12,2428253294,Нет,user
5853,17.10.2024 12:31:59,2329250131,Нет,Видео
5854,17.10.2024 13:13:29,2208135221,Нет,МП


In [ ]:
df['ID устройства'] = df['ID устройства'].astype(str)
df['Серийный номер'] = df['Серийный номер'].apply(lambda x: x.split(sep=','))
df2 = df.explode('Серийный номер')
df2 = df2.reset_index().drop('index', axis=1)
df2['Серийный номер'].apply(lambda x: x.strip())
df2['Отметка времени'] = pd.to_datetime(df2['Отметка времени'],dayfirst=True)
df2['counter'] = df2.groupby('Отметка времени').cumcount()
df2['Отметка времени'] = df2['Отметка времени'] + pd.to_timedelta(df2['counter'], unit='s')
df3 = df2
list_of_columns = ['ID устройства', 'Обращение по:']
for i in list_of_columns:
    df3[i] = df3[i].fillna('').astype(str).apply(lambda x:x.split(','))
    df3 = df3.explode(i)
    df3[i] = df3[i].apply(lambda x:x.strip())
df3.to_csv(TEMP_FILENAME) 
get_s3_instance()
upload_dump_to_s3()
remove_temp_files()
requests.get(os.getenv("URL_ALERT"))
return {
    'statusCode': 200,
    'body': 'Hello World!',
}